

# iMaterialist Challenge (Furniture) at FGVC5

### TFNW Kaggle Team

#### Train a ResNet50 network

https://www.kaggle.com/c/imaterialist-challenge-furniture-2018

@alkari

*Restart runtime*

In [0]:
#!kill -9 -1

Check GPU/Memory

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print('Gen RAM Free: ' + humanize.naturalsize( psutil.virtual_memory().available ), ' I Proc size: ' + humanize.naturalsize( process.memory_info().rss))
 print('GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB'.format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Install pre-requisites

In [0]:
! pip install --upgrade -q pydot

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

# Start here...

In [0]:
# Helper functions
def elapsed (start):
    """
    Returns elapsed time in hh:mm:ss format from start time in unix format
    """
    elapsed = time.time()-start
    hours, rem = divmod(elapsed, 3600)
    minutes, seconds = divmod(rem, 60)
    return("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [0]:
import os
os.chdir('/content/drive')

In [20]:
import time
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [21]:
# Load Classes

test_path="iMaterialist/validation_dataset/"
test_dataset_name = 'validation_last'
with h5py.File(test_path+'{}_labels.h5'.format(test_dataset_name), 'r') as hf:
    test_set_y_orig = np.array(hf['{}_labels'.format(test_dataset_name)][:])

classes = []
for i in range (1,len(test_set_y_orig)):
    if test_set_y_orig[i] not in classes:
        classes.append(test_set_y_orig[i])      
classes = np.array(classes) # the list of classes
print(classes.shape)
test_set_y_orig = None

(128,)


In [0]:
def load_dataset(train_path, train_dataset_name, batch_size=1000):
    #train_path="iMaterialist/train_dataset/"
    #test_path="iMaterialist/validation_dataset/"
    #train_dataset_name = 'train_1'
    #test_dataset_name = 'validation_last'


    # Train dataset
    with h5py.File(train_path+'{}_images.h5'.format(train_dataset_name), 'r') as hf:
        train_set_x_orig = np.array(hf['{}_images'.format(train_dataset_name)][batch_size-1000:batch_size])   
    with h5py.File(train_path+'{}_labels.h5'.format(train_dataset_name), 'r') as hf:
        train_set_y_orig = np.array(hf['{}_labels'.format(train_dataset_name)][batch_size-1000:batch_size])

    # Test dataset (validation)
    #with h5py.File(test_path+'{}_images.h5'.format(test_dataset_name), 'r') as hf:
    #    test_set_x_orig = np.array(hf['{}_images'.format(test_dataset_name)][batch_size-1000:batch_size])   
    #with h5py.File(test_path+'{}_labels.h5'.format(test_dataset_name), 'r') as hf:
    #    test_set_y_orig = np.array(hf['{}_labels'.format(test_dataset_name)][batch_size-1000:batch_size])

    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    #test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    #return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes
    return train_set_x_orig, train_set_y_orig, classes


In [4]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          13029         529       11840           0         659       12283
Swap:             0           0           0


In [0]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of ResNet identity block
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path
    X = Conv2D(filters= F2, kernel_size= (f,f), strides= (1,1), padding= 'same', name= conv_name_base + '2b', kernel_initializer= glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis= 3, name= bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters= F3, kernel_size= (1,1), strides= (1,1), padding= 'valid', name= conv_name_base + '2c', kernel_initializer= glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis= 3, name= bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    
    return X

In [0]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of ResNet convolutional block
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path 

    X = Conv2D(F2, (f,f), strides= (1,1), padding= 'same', name= conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis= 3, name= bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path 
    X = Conv2D(F3, (1,1), strides=(1,1), padding='valid', name= conv_name_base +'2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis= 3, name= bn_name_base +'2c')(X)

    ##### SHORTCUT PATH #### 
    
    X_shortcut = Conv2D(F3, (1,1), strides=(s,s), padding='valid', name= conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut) 
    X_shortcut = BatchNormalization(axis = 3, name= bn_name_base +'1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    
    return X

In [0]:
def ResNet50(input_shape = (300, 300, 3), classes = 129):
    """
    Implementation of ResNet50 with the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)


    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = AveragePooling2D(pool_size=(2, 2))(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)


    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

### Build the model

In [0]:
model = ResNet50(input_shape = (300, 300, 3), classes = 129)

### Compile the model

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

The model is now ready to be trained. The only thing you need is a dataset.

In [0]:
    train_path="iMaterialist/train_dataset/"
    test_path="iMaterialist/validation_dataset/"
    #train_dataset_name = 'train_2'
    test_dataset_name = 'validation_last'
    load_batch_size = 1000
    assert load_batch_size == 1000


# Begin training

In [0]:
start = time.time()
for dataset_number in range (1,10):
  train_dataset_name = "train_{}".format(dataset_number)
  for batch in range(load_batch_size,5001,load_batch_size):
    X_train_orig, Y_train_orig, classes = load_dataset(train_path, train_dataset_name, batch)

    # Normalize image vectors
    X_train = X_train_orig/255.
    #X_test = X_test_orig/255.

    # Convert training and test labels to one hot matrices
    Y_train = convert_to_one_hot(Y_train_orig, 129).T
    #Y_test = convert_to_one_hot(Y_test_orig, 129).T

    #print ("number of training examples = " + str(X_train.shape[0]))
    #print ("number of test examples = " + str(X_test.shape[0]))
    #print ("X_train shape: " + str(X_train.shape))
    #print ("Y_train shape: " + str(Y_train.shape))
    #print ("X_test shape: " + str(X_test.shape))
    #print ("Y_test shape: " + str(Y_test.shape))

    print('\nTraining dataset {}'.format(dataset_number))
    print("\n*****Training batch# {}".format(batch)+"*****\n")
    model.fit(X_train, Y_train, epochs = 20, batch_size = 50)
    print('\n-------------------------- Elapsed time: {} --------------------------'.format(elapsed(start)))
    model.save('iMaterlialist-Keras-ResNet50-{}.h5'.format(dataset_number))
    print('\nCheckpoint saved. Elapsed time: {}'.format(elapsed(start)))


Training dataset 1

*****Training batch# 1000*****

Epoch 1/20
1000/1000 [==============================] - 58s 58ms/step - loss: 14.5727 - acc: 0.0110
Epoch 2/20
1000/1000 [==============================] - 50s 50ms/step - loss: 10.7834 - acc: 0.0120
Epoch 3/20
1000/1000 [==============================] - 50s 50ms/step - loss: 5.2579 - acc: 0.0170
Epoch 4/20
1000/1000 [==============================] - 50s 50ms/step - loss: 5.2729 - acc: 0.0320
Epoch 5/20
1000/1000 [==============================] - 50s 50ms/step - loss: 5.4402 - acc: 0.0400
Epoch 6/20
1000/1000 [==============================] - 50s 50ms/step - loss: 4.9388 - acc: 0.0680
Epoch 7/20
1000/1000 [==============================] - 50s 50ms/step - loss: 4.5930 - acc: 0.0910
Epoch 8/20
1000/1000 [==============================] - 50s 50ms/step - loss: 4.5367 - acc: 0.1250
Epoch 9/20
 850/1000 [========================>.....] - ETA: 7s - loss: 4.2452 - acc: 0.1600

1000/1000 [==============================] - 50s 50ms/step - loss: 4.1806 - acc: 0.1620
Epoch 10/20
1000/1000 [==============================] - 50s 50ms/step - loss: 3.5994 - acc: 0.2040
Epoch 11/20
1000/1000 [==============================] - 50s 50ms/step - loss: 3.3879 - acc: 0.2830
Epoch 12/20
1000/1000 [==============================] - 50s 50ms/step - loss: 3.2707 - acc: 0.2890
Epoch 13/20
1000/1000 [==============================] - 49s 49ms/step - loss: 2.9728 - acc: 0.3440
Epoch 14/20
1000/1000 [==============================] - 49s 49ms/step - loss: 2.7992 - acc: 0.4000
Epoch 15/20
1000/1000 [==============================] - 50s 50ms/step - loss: 2.6742 - acc: 0.4170
Epoch 16/20
1000/1000 [==============================] - 50s 50ms/step - loss: 2.5705 - acc: 0.4510
Epoch 17/20
1000/1000 [==============================] - 50s 50ms/step - loss: 2.2129 - acc: 0.5090
Epoch 18/20
 500/1000 [==============>...............] - ETA: 24s - loss: 1.9072 - acc: 0.6020

1000/1000 [==============================] - 49s 49ms/step - loss: 1.9310 - acc: 0.5900
Epoch 19/20
1000/1000 [==============================] - 50s 50ms/step - loss: 2.0312 - acc: 0.6030
Epoch 20/20
1000/1000 [==============================] - 50s 50ms/step - loss: 2.9689 - acc: 0.4750

Training dataset 1

*****Training batch# 2000*****

Epoch 1/20
1000/1000 [==============================] - 50s 50ms/step - loss: 5.3230 - acc: 0.0510
Epoch 2/20
1000/1000 [==============================] - 50s 50ms/step - loss: 4.7631 - acc: 0.0750
Epoch 3/20
1000/1000 [==============================] - 50s 50ms/step - loss: 4.3647 - acc: 0.1200
Epoch 4/20
1000/1000 [==============================] - 50s 50ms/step - loss: 3.7927 - acc: 0.2040
Epoch 5/20
1000/1000 [==============================] - 49s 49ms/step - loss: 3.0923 - acc: 0.3260
Epoch 6/20
1000/1000 [==============================] - 50s 50ms/step - loss: 2.5727 - acc: 0.4520
Epoch 7/20
 150/1000 [===>..........................] - ETA: 43s 

1000/1000 [==============================] - 50s 50ms/step - loss: 2.4389 - acc: 0.4890
Epoch 8/20
1000/1000 [==============================] - 49s 49ms/step - loss: 2.0799 - acc: 0.5840
Epoch 9/20
1000/1000 [==============================] - 49s 49ms/step - loss: 1.6162 - acc: 0.6740
Epoch 10/20
1000/1000 [==============================] - 50s 50ms/step - loss: 1.1495 - acc: 0.7770
Epoch 11/20
1000/1000 [==============================] - 50s 50ms/step - loss: 1.0662 - acc: 0.8220
Epoch 12/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.8806 - acc: 0.8520
Epoch 13/20
1000/1000 [==============================] - 49s 49ms/step - loss: 0.7943 - acc: 0.8830
Epoch 14/20
1000/1000 [==============================] - 49s 49ms/step - loss: 0.7391 - acc: 0.9060
Epoch 15/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.6238 - acc: 0.9000
Epoch 16/20
 450/1000 [============>.................] - ETA: 27s - loss: 0.5829 - acc: 0.9356

1000/1000 [==============================] - 50s 50ms/step - loss: 0.5628 - acc: 0.9350
Epoch 17/20
1000/1000 [==============================] - 49s 49ms/step - loss: 0.5739 - acc: 0.9260
Epoch 18/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.5195 - acc: 0.9470
Epoch 19/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.5422 - acc: 0.9500
Epoch 20/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.7085 - acc: 0.9110

Training dataset 1

*****Training batch# 3000*****

Epoch 1/20
1000/1000 [==============================] - 50s 50ms/step - loss: 6.3604 - acc: 0.0570
Epoch 2/20
1000/1000 [==============================] - 50s 50ms/step - loss: 4.6827 - acc: 0.0920
Epoch 3/20
1000/1000 [==============================] - 50s 50ms/step - loss: 4.8324 - acc: 0.1070
Epoch 4/20
1000/1000 [==============================] - 49s 49ms/step - loss: 4.2704 - acc: 0.1720
Epoch 5/20
 200/1000 [=====>........................] - ETA: 39

...

1000/1000 [==============================] - 49s 49ms/step - loss: 0.2916 - acc: 0.9400
Epoch 6/20
1000/1000 [==============================] - 48s 48ms/step - loss: 0.1032 - acc: 0.9860
Epoch 7/20
1000/1000 [==============================] - 49s 49ms/step - loss: 0.0488 - acc: 0.9930
Epoch 8/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0307 - acc: 0.9990
Epoch 9/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0218 - acc: 0.9970
Epoch 10/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0186 - acc: 0.9980
Epoch 11/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0125 - acc: 0.9990
Epoch 12/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0175 - acc: 0.9960
Epoch 13/20
1000/1000 [==============================] - 49s 49ms/step - loss: 0.0073 - acc: 1.0000
Epoch 14/20
 450/1000 [============>.................] - ETA: 26s - loss: 0.0048 - acc: 1.0000

1000/1000 [==============================] - 49s 49ms/step - loss: 0.0055 - acc: 1.0000
Epoch 15/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0067 - acc: 1.0000
Epoch 16/20
1000/1000 [==============================] - 49s 49ms/step - loss: 0.0041 - acc: 1.0000
Epoch 17/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0050 - acc: 1.0000
Epoch 18/20
1000/1000 [==============================] - 49s 49ms/step - loss: 0.0031 - acc: 1.0000
Epoch 19/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0076 - acc: 0.9990
Epoch 20/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0031 - acc: 1.0000

Training dataset 4

*****Training batch# 2000*****

Epoch 1/20
1000/1000 [==============================] - 50s 50ms/step - loss: 5.6116 - acc: 0.1250
Epoch 2/20
1000/1000 [==============================] - 50s 50ms/step - loss: 3.0012 - acc: 0.3170
Epoch 3/20
 300/1000 [========>.....................] - ETA: 

1000/1000 [==============================] - 50s 50ms/step - loss: 1.7910 - acc: 0.5610
Epoch 4/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.7493 - acc: 0.8240
Epoch 5/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.2271 - acc: 0.9590
Epoch 6/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0807 - acc: 0.9900
Epoch 7/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0495 - acc: 0.9940
Epoch 8/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0272 - acc: 0.9960
Epoch 9/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0461 - acc: 0.9920
Epoch 10/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0159 - acc: 0.9980
Epoch 11/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0146 - acc: 0.9980
Epoch 12/20
 500/1000 [==============>...............] - ETA: 24s - loss: 0.0140 - acc: 0.9960

1000/1000 [==============================] - 49s 49ms/step - loss: 0.0116 - acc: 0.9980
Epoch 13/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0177 - acc: 0.9980
Epoch 14/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0106 - acc: 0.9980
Epoch 15/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0111 - acc: 0.9990
Epoch 16/20
1000/1000 [==============================] - 49s 49ms/step - loss: 0.0077 - acc: 0.9980
Epoch 17/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0044 - acc: 0.9990
Epoch 18/20
1000/1000 [==============================] - 49s 49ms/step - loss: 0.0101 - acc: 0.9990
Epoch 19/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0049 - acc: 1.0000
Epoch 20/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.0108 - acc: 0.9980

Training dataset 4

*****Training batch# 3000*****

Epoch 1/20
 450/1000 [============>.................] - ETA

1000/1000 [==============================] - 49s 49ms/step - loss: 5.4970 - acc: 0.1290
Epoch 2/20
1000/1000 [==============================] - 50s 50ms/step - loss: 2.9540 - acc: 0.3270
Epoch 3/20
1000/1000 [==============================] - 50s 50ms/step - loss: 1.7252 - acc: 0.5890
Epoch 4/20
1000/1000 [==============================] - 49s 49ms/step - loss: 0.7442 - acc: 0.8260
Epoch 5/20
1000/1000 [==============================] - 50s 50ms/step - loss: 0.2302 - acc: 0.9580
Epoch 6/20
 850/1000 [========================>.....] - ETA: 7s - loss: 0.0680 - acc: 0.9906

In [0]:
model.save('iMaterlialist-Keras-ResNet50.h5')

In [0]:
#model.fit(X_train, Y_train, epochs = 20, batch_size = 64)

### Try this model on the test set.

In [0]:
model = load_model('iMaterlialist-Keras-ResNet50-3.h5') 

In [0]:
def load_test_dataset(test_path, test_dataset_name):

    # Test dataset (validation)
    with h5py.File(test_path+'{}_images.h5'.format(test_dataset_name), 'r') as hf:
        test_set_x_orig = np.array(hf['{}_images'.format(test_dataset_name)][:500])   
    with h5py.File(test_path+'{}_labels.h5'.format(test_dataset_name), 'r') as hf:
        test_set_y_orig = np.array(hf['{}_labels'.format(test_dataset_name)][:500])

    
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    X_test = test_set_x_orig/255.
    Y_test = convert_to_one_hot(test_set_y_orig, 129).T
    
    return X_test, Y_test


test_path="iMaterialist/validation_dataset/"
test_dataset_name = 'validation_last'

X_test, Y_test = load_test_dataset(test_path, test_dataset_name)

In [24]:
start = time.time()
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))
print('\nElapsed time: {}'.format(elapsed(start)))

500/500 [==============================] - 453s 906ms/step
Loss = 8.32625260925293
Test Accuracy = 0.10600000002980232

Elapsed time: 00:07:32.90


## Test on your own image

You can upload an image and see the output of the model. To do this:
    1. Click on "File" in the upper bar of this notebook, then click "Open".
    2. Add your image to this Jupyter Notebook's directory
    3. Write your image's name in the following code
    4. Predict! 

In [0]:
img_path = 'my_image.jpg'
img = image.load_img(img_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print('Input image shape:', x.shape)
my_image = scipy.misc.imread(img_path)
imshow(my_image)
print("class prediction = ")
print(model.predict(x))

You can also print a summary of your model by running the following code.

In [0]:
model.summary()

Visualize this ResNet50. You can also download a .png picture of your model by going to "File -> Open...-> model.png".

In [0]:
plot_model(model, to_file='model.png')
SVG(model_to_dot(model).create(prog='dot', format='svg'))

### References 

This notebook presents the ResNet algorithm due to He et al. (2015). The implementation here also took significant inspiration and follows the structure given in the github repository of Francois Chollet: 

- Coursera Convolusional Neural Networks - deeplearning.ai
- Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun - [Deep Residual Learning for Image Recognition (2015)](https://arxiv.org/abs/1512.03385)
- Francois Chollet's github repository: https://github.com/fchollet/deep-learning-models/blob/master/resnet50.py


In [0]:
! rm -rf /swapfile
#!dd if=/dev/zero of=/swapfile bs=1G count=20
!fallocate -l 20G /swapfile
!chmod 600 /swapfile
!ls -lh /swapfile
!mkswap /swapfile
!swapon /swapfile
!sysctl vm.swappiness=10
!sysctl vm.vfs_cache_pressure=60
!swapon -s
!free -m